## Bracket Challenge Predictor

Reading data

In [393]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re

from functools import reduce
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from xgboost import XGBRegressor

## Data Preprocessing

In [727]:
team_matchups = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Tournament Matchups.csv")

away_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Away.csv")
home_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Home.csv")
neutral_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Neutral.csv")

conf_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Conference Results.csv")

team_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Team Results.csv")
team_hist = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Resumes.csv")
team_v_ranked = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/TeamRankings.csv")
team_mis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/KenPom Barttorvik.csv")

ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")

cur_coaches = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ Current NCAAM Coaches.csv")
coach_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Coach Results.csv")

seed_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Seed Results.csv")
upset_count = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Count.csv")
upset_inf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Seed Info.csv")

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/2085953661.py:14: DtypeWarning: Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.
  ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")


since ken_ datasets don't have TEAMNO var will first create seperate lookup table to create a column for its respective id

In [728]:
team_ids = team_matchups[['YEAR','TEAM','TEAM NO']].drop_duplicates()

team_res.rename(columns={'TEAM ID':'TEAM NO'},inplace=True)

In [729]:
temp_away = away_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_home = home_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_neutral = neutral_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]

#adds _A to all columns to represent Away
temp_away.columns = list(temp_away.columns[:2]) + [col + "_A" for col in temp_away.columns[2:]]

#_H for Home
temp_home.columns = list(temp_home.columns[:2]) + [col + "_H" for col in temp_home.columns[2:]]

#_N for Neutral
temp_neutral.columns = list(temp_neutral.columns[:2]) + [col + "_N" for col in temp_neutral.columns[2:]]


#combined stats as they are highly correlated away, home, and neutral
#function to create new combined dataframe easily
def var_mean(dfs, vars):
    combined_df = [dfs[0].loc[:,vars],dfs[1].loc[:,vars],dfs[2].loc[:,vars]]
    combined_df = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),combined_df)
    for var in vars[2:]:
        combined_df[var] = combined_df[[var+'_x',var+'_y',var]].mean(axis=1)
        combined_df.drop([var+'_x',var+'_y'],axis=1,inplace=True)
    return combined_df

vars = ['YEAR','TEAM NO','BADJ EM','FTR','FTRD',
        'BADJ T','TOV%','TOV%D','OREB%','OP OREB%','RAW T','WAB',
        '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%',
        'AST%', 'OP AST%', '2PTR', '3PTR', '2PTRD', '3PTRD','ELITE SOS']
dfs = [away_stats,home_stats,neutral_stats]

team_stats = var_mean(dfs,vars)

temp_res = team_res.loc[:,['TEAM','PAKE','PASE','R64','R32','S16','E8','F4',
                           'F2','CHAMP','TOP2']]

temp_his = team_hist.drop(['TEAM','SEED','ROUND','Q1 PLUS Q2 W','B POWER','BID TYPE'],axis=1)

temp_v_ranked = team_v_ranked.loc[:,['YEAR','TEAM NO','TR RATING','V 1-25 WINS','V 1-25 LOSS',
                                     'V 26-50 WINS','V 26-50 LOSS','LUCK RATING','CONSISTENCY TR RATING']]

temp_mis = team_mis.loc[:,['YEAR','CONF','CONF ID','TEAM','TEAM NO','K OFF','K DEF','AVG HGT','EFF HGT','EXP','TALENT']]


Creating supplmental dataset that aren't dependent on team metrics (i.e: conference, coach, upsets)

For conference we'll just use the conf_perf dataset with selected variables

In [415]:
conf_perf = conf_perf.drop(['GAMES','W','L'],axis=1)

For coaches will create a column in coach_perf on the years and Teams they've coaches respectively

In [683]:
ken_conf.rename(columns={'Season':'YEAR','Full Team Name':'TEAM'},inplace=True)

#filtering data to match years in other datasets
coach_stats = ken_conf[(ken_conf['YEAR']<=2024) & (ken_conf['YEAR']>=2008) & (ken_conf['YEAR'] != 2020)]

#dictionary to remap Team Name is consistent form
team_name_mapping = {
    'Abilene Christian Wildcats': 'Abilene Christian',
    'Akron Zips': 'Akron',
    'Alabama Crimson Tide': 'Alabama',
    'Albany Great Danes': 'Albany',
    'American Eagles': 'American',
    'Arizona Wildcats': 'Arizona',
    'Arizona State Sun Devils': 'Arizona St.',
    'Arkansas Razorbacks': 'Arkansas',
    'Arkansas–Pine Bluff Golden Lions': 'Arkansas Pine Bluff',
    'Auburn Tigers': 'Auburn',
    'Austin Peay Governors': 'Austin Peay',
    'BYU Cougars': 'BYU',
    'Baylor Bears': 'Baylor',
    'Belmont Bruins': 'Belmont',
    'Binghamton Bearcats': 'Binghamton',
    'Boise State Broncos': 'Boise St.',
    'Boston College Eagles': 'Boston College',
    'Boston University Terriers': 'Boston University',
    'Bradley Braves': 'Bradley',
    'Bucknell Bison': 'Bucknell',
    'Buffalo Bulls': 'Buffalo',
    'Butler Bulldogs': 'Butler',
    'Cal Poly Mustangs': 'Cal Poly',
    'Cal State Bakersfield Roadrunners': 'Cal St. Bakersfield',
    'Cal State Fullerton Titans': 'Cal St. Fullerton',
    'Cal State Northridge Matadors': 'Cal St. Northridge',
    'California Golden Bears': 'California',
    'Chattanooga Mocs': 'Chattanooga',
    'Cincinnati Bearcats': 'Cincinnati',
    'Clemson Tigers': 'Clemson',
    'Cleveland State Vikings': 'Cleveland St.',
    'Coastal Carolina Chanticleers': 'Coastal Carolina',
    'Colgate Raiders': 'Colgate',
    'Charleston Cougars': 'College of Charleston',
    'Colorado Buffaloes': 'Colorado',
    'Colorado State Rams': 'Colorado St.',
    'UConn Huskies': 'Connecticut',
    'Cornell Big Red': 'Cornell',
    'Creighton Bluejays': 'Creighton',
    'Davidson Wildcats': 'Davidson',
    'Dayton Flyers': 'Dayton',
    "Delaware Fightin' Blue Hens": 'Delaware',
    'Detroit Mercy Titans': 'Detroit',
    'Drake Bulldogs': 'Drake',
    'Drexel Dragons': 'Drexel',
    'Duke Blue Devils': 'Duke',
    'Duquesne Dukes': 'Duquesne',
    'East Tennessee State Buccaneers': 'East Tennessee St.',
    'Eastern Kentucky Colonels': 'Eastern Kentucky',
    'Eastern Washington Wagles': 'Eastern Washington',
    'Fairleigh Dickinson Knights': 'Fairleigh Dickinson',
    'Florida Gators': 'Florida',
    'Florida Atlantic Owls': 'Florida Atlantic',
    'Florida Gulf Coast Eagles': 'Florida Gulf Coast',
    'Florida State Seminoles': 'Florida St.',
    'Fresno Stata Bulldogs': 'Fresno St.',
    'Furman Paladins': 'Furman',
    "Gardner–Webb Runnin' Bulldogs": 'Gardner Webb',
    'George Mason Patriots': 'George Mason',
    'George Washington Revolutionaries': 'George Washington',
    'Georgetown Hoyas': 'Georgetown',
    'Georgia Bulldogs': 'Georgia',
    'Georgia State Panthers': 'Georgia St.',
    'Georgia Tech Yellow Jackets': 'Georgia Tech',
    'Gonzaga Bulldogs': 'Gonzaga',
    'Grambling State Tigers': 'Grambling St.',
    'Grand Canyon Antelopes': 'Grand Canyon',
    'Green Bay Phoenix': 'Green Bay',
    'Hampton Pirates': 'Hampton',
    'Harvard Crimson': 'Harvard',
    'Hawaii Rainbow Warriors': 'Hawaii',
    'Holy Cross Crusaders': 'Holy Cross',
    'Houston Cougars': 'Houston',
    'Howard Bison': 'Howard',
    'Illinois Fighting Illini': 'Illinois',
    'Indiana Hoosiers': 'Indiana',
    'Indiana State Sycamores': 'Indiana St.',
    'Iona Gaels': 'Iona',
    'Iowa Hawkeyes': 'Iowa',
    'Iowa State Cyclones': 'Iowa St.',
    'Jacksonville State Tigers': 'Jacksonville St.',
    'James Madison Dukes': 'James Madison',
    'Kansas Jayhawks': 'Kansas',
    'Kansas State Wildcats': 'Kansas St.',
    'Kennesaw State Owls': 'Kennesaw St.',
    'Kent State Golden Flashes': 'Kent St.',
    'Kentucky Wildcats': 'Kentucky',
    'LIU Sharks': 'LIU Brooklyn',
    'LSU Tigers': 'LSU',
    'La Salle Explorers': 'La Salle',
    'Lafayette Leopards': 'Lafayette',
    'Lehigh Mountain Hawks': 'Lehigh',
    'Liberty Flames': 'Liberty',
    'Lipscomb Bisons': 'Lipscomb',
    'Little Rock Trojans': 'Little Rock',
    'Long Beach State Beach': 'Long Beach St.',
    'Longwood Lancers': 'Longwood',
    "Louisiana Ragin' Cajuns": 'Louisiana Lafayette',
    'Louisville Cardinals': 'Louisville',
    'Loyola Chicago Ramblers': 'Loyola Chicago',
    'Loyola (Maryland) Greyhounds': 'Loyola MD',
    'Manhattan Jaspers': 'Manhattan',
    'Marquette Golden Eagles': 'Marquette',
    'Marshall Thundering Herd': 'Marshall',
    'Maryland Terrapins': 'Maryland',
    'UMass Minutemen': 'Massachusetts',
    'McNeese Cowboys': 'McNeese St.',
    'Memphis Tigers': 'Memphis',
    'Mercer Bears': 'Mercer',
    'Miami Hurricanes': 'Miami FL',
    'Michigan Wolverines': 'Michigan',
    'Michigan State Spartans': 'Michigan St.',
    'Middle Tennessee Blue Raiders': 'Middle Tennessee',
    'Milwaukee Panthers': 'Milwaukee',
    'Minnesota Golden Gophers': 'Minnesota',
    'Ole Miss Rebels': 'Mississippi',
    'Mississippi State Bulldogs': 'Mississippi St.',
    'Mississippi Valley State Delta Devils': 'Mississippi Valley St.',
    'Missouri Tigers': 'Missouri',
    'Montana Grizzlies': 'Montana',
    'Montana State Bobcats': 'Montana St.',
    'Morehead State Eagles': 'Morehead St.',
    'Morgan State Bears': 'Morgan St.',
    "Mount St. Mary's Mountaineers": "Mount St. Mary's",
    'Murray State Racers': 'Murray St.',
    'Nebraska Cornhuskers': 'Nebraska',
    'Nevada Wolf Pack': 'Nevada',
    'New Mexico Lobos': 'New Mexico',
    'New Mexico State Aggies': 'New Mexico St.',
    'Norfolk State Spartans': 'Norfolk St.',
    'North Carolina Tar Heels': 'North Carolina',
    'North Carolina A&T Aggies': 'North Carolina A&T',
    'North Carolina Central Eagles': 'North Carolina Central',
    'NC State Wolfpack': 'North Carolina St.',
    'North Dakota Fighting Hawks': 'North Dakota',
    'North Dakota State Bison': 'North Dakota St.',
    'North Texas Mean Green': 'North Texas',
    'Northeastern Huskies': 'Northeastern',
    'Northern Colorado Bears': 'Northern Colorado',
    'Northern Iowa Panthers': 'Northern Iowa',
    'Northern Kentucky Norse': 'Northern Kentucky',
    'Northwestern Wildcats': 'Northwestern',
    'Northwestern State Demons': 'Northwestern St.',
    'Notre Dame Fighting Irish': 'Notre Dame',
    'Oakland Golden Grizzlies': 'Oakland',
    'Ohio Bobcats': 'Ohio',
    'Ohio State Buckeyes': 'Ohio St.',
    'Oklahoma Sooners': 'Oklahoma',
    'Oklahoma State Cowboys': 'Oklahoma St.',
    'Old Dominion Monarchs': 'Old Dominion',
    'Oral Roberts Golden Eagles': 'Oral Roberts',
    'Oregon Ducks': 'Oregon',
    'Oregon State Beavers': 'Oregon St.',
    'Pacific Tigers': 'Pacific',
    'Penn Quakers': 'Penn',
    'Penn State Nittany Lions': 'Penn St.',
    'Pittsburgh Panthers': 'Pittsburgh',
    'Portland State Vikings': 'Portland St.',
    'Princeton Tigers': 'Princeton',
    'Providence Friars': 'Providence',
    'Purdue Boilermakers': 'Purdue',
    'Radford Highlanders': 'Radford',
    'Rhode Island Rams': 'Rhode Island',
    'Richmond Spiders': 'Richmond',
    'Robert Morris Colonials': 'Robert Morris',
    'Rutgers Scarlet Knights': 'Rutgers',
    'SMU Mustangs': 'SMU',
    "Saint Joseph's Hawks": "Saint Joseph's",
    'Saint Louis Billikens': 'Saint Louis',
    "Saint Mary's Gaels": "Saint Mary's",
    "Saint Peter's Peacocks": "Saint Peter's",
    'Sam Houston Bearkats': 'Sam Houston St.',
    'Samford Bulldogs': 'Samford',
    'San Diego Toreros': 'San Diego',
    'San Diego State Aztecs': 'San Diego St.',
    'San Francisco Dons': 'San Francisco',
    'Seton Hall Pirates': 'Seton Hall',
    'Siena Saints': 'Siena',
    'South Alabama Jaguars': 'South Alabama',
    'South Carolina Gamecocks': 'South Carolina',
    'South Dakota State Bulldogs': 'South Dakota St.',
    'South Florida Bulls': 'South Florida',
    'Southern Jaguars': 'Southern',
    'Southern Miss Golden Eagles': 'Southern Miss',
    'St. Bonaventure Bonnies': 'St. Bonaventure',
    "St. John's Red Storm": "St. John's",
    'Stanford Cardinal': 'Stanford',
    'Stephen F. Austin Lumberjacks': 'Stephen F. Austin',
    'Stetson Hatters': 'Stetson',
    'Stony Brook Seawolves': 'Stony Brook',
    'Syracuse Orange': 'Syracuse',
    'TCU Horned Frogs': 'TCU',
    'Temple Owls': 'Temple',
    'Tennessee Volunteers': 'Tennessee',
    'Texas Longhorns': 'Texas',
    'Texas A&M Aggies': 'Texas A&M',
    'Texas A&M-Corpus Chris Islanders': 'Texas A&M Corpus Chris',
    'Texas Southern Tigers': 'Texas Southern',
    'Texas Tech Red Raiders': 'Texas Tech',
    'Troy Trojans': 'Troy',
    'Tulsa Golden Hurricane': 'Tulsa',
    'UAB Blazers': 'UAB',
    'UC Davis Aggies': 'UC Davis',
    'UC Irvine Anteaters': 'UC Irvine',
    'UC Santa Barbara Gauchos': 'UC Santa Barbara',
    'UCF Knights': 'UCF',
    'UCLA Bruins': 'UCLA',
    'UMBC Retrievers': 'UMBC',
    'UNC Asheville Bulldogs': 'UNC Asheville',
    'UNC Greensboro Spartans': 'UNC Greensboro',
    'UNC Wilmington Seahawks': 'UNC Wilmington',
    "UNLV Runnin' Rebels": 'UNLV',
    'USC Trojans': 'USC',
    'UT Arlington Mavericks': 'UT Arlington',
    'UTEP Miners': 'UTEP',
    'UTSA Roadrunners': 'UTSA',
    'Utah Utes': 'Utah',
    'Utah State Aggies': 'Utah St.',
    'VCU Rams': 'VCU',
    'Valparaiso Beacons': 'Valparaiso',
    'Vanderbilt Commodores': 'Vanderbilt',
    'Vermont Catamounts': 'Vermont',
    'Villanova Wildcats': 'Villanova',
    'Virginia Cavaliers': 'Virginia',
    'Virginia Tech Hokies': 'Virginia Tech',
    'Wagner Seahawks': 'Wagner',
    'Wake Forest Demon Deacons': 'Wake Forest',
    'Washington Huskies': 'Washington',
    'Washington State Cougars': 'Washington St.',
    'Weber State Wildcats': 'Weber St.',
    'West Virginia Mountaineers': 'West Virginia',
    'Western Kentucky Hilltoppers': 'Western Kentucky',
    'Western Michigan Broncos': 'Western Michigan',
    'Wichita State Shockers': 'Wichita St.',
    'Winthrop Eagles': 'Winthrop',
    'Wisconsin Badgers': 'Wisconsin',
    'Wofford Terriers': 'Wofford',
    'Wright State Raiders': 'Wright St.',
    'Wyoming Cowboys': 'Wyoming',
    'Xavier Musketeers': 'Xavier',
    'Yale Bulldogs': 'Yale'
}

#need to manually add Hartford Hawks as not in dataset
hart = {'YEAR': 2021,
        'TEAM': 'Hartford'}

coach_stats['TEAM'] = coach_stats['TEAM'].replace(team_name_mapping)

coach_stats = pd.concat([coach_stats,pd.DataFrame([hart])],ignore_index=True)

#now merging the TEAM NO to this dataset
coach_stats = coach_stats.loc[:,['YEAR','TEAM']].merge(team_ids,on=['YEAR','TEAM'],how='left')

#removing nan on TEAM NO as that means they weren't in the tournament that year
coach_stats = coach_stats.dropna(subset=['TEAM NO'])

coach_stats.head()

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/3477688823.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coach_stats['TEAM'] = coach_stats['TEAM'].replace(team_name_mapping)


,YEAR,TEAM,TEAM NO
0,2015,Kentucky,510.0
1,2021,Gonzaga,856.0
2,2024,Connecticut,1067.0
3,2008,Kansas,43.0
4,2019,Virginia,745.0


Now need to add columns on years coached, for what team to then merge on 'coach_per' statisics which are of interest

Combining two rows since there was a typo and their the same person

In [684]:
#converting F4% and CHAMP% as a int
coach_perf["F4%"] = coach_perf["F4%"].str.rstrip("%").astype(float) / 100
coach_perf["CHAMP%"] = coach_perf["CHAMP%"].str.rstrip("%").astype(float) / 100


steve_prohm = {
    "COACH ID": min(coach_perf.iloc[72,0],coach_perf.iloc[187,0],coach_perf.iloc[291,0]),  # Keep the lower ID
    "COACH": "Steve Prohm",  # Corrected name
    "PAKE": coach_perf.loc[[72, 187, 291], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[72, 187, 291], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[72, 187, 291], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[72, 187, 291], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[72, 187, 291], "GAMES"].sum(),
    "W": coach_perf.loc[[72, 187, 291], "W"].sum(),
    "L": coach_perf.loc[[72, 187, 291], "L"].sum(),
    "WIN%": coach_perf.loc[[72, 187, 291], "WIN%"].mean(),
    "R64": coach_perf.loc[[72, 187, 291], "R64"].sum(),
    "R32": coach_perf.loc[[72, 187, 291], "R32"].sum(),
    "S16": coach_perf.loc[[72, 187, 291], "S16"].sum(),
    "E8": coach_perf.loc[[72, 187, 291], "E8"].sum(),
    "F4": coach_perf.loc[[72, 187, 291], "F4"].sum(),
    "F2": coach_perf.loc[[72, 187, 291], "F2"].sum(),
    "CHAMP": coach_perf.loc[[72, 187, 291], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[72, 187, 291], "TOP2"].sum(),
    "F4%": coach_perf.loc[[72, 187, 291], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[72, 187, 291], "CHAMP%"].mean(),
}

bob_marlin = {
    "COACH ID": min(coach_perf.iloc[164,0],coach_perf.iloc[215,0]),  # Keep the lower ID
    "COACH": "Bob Marlin",  # Corrected name
    "PAKE": coach_perf.loc[[164, 215], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[164, 215], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[164, 215], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[164, 215], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[164, 215], "GAMES"].sum(),
    "W": coach_perf.loc[[164, 215], "W"].sum(),
    "L": coach_perf.loc[[164, 215], "L"].sum(),
    "WIN%": coach_perf.loc[[164, 215], "WIN%"].mean(),
    "R64": coach_perf.loc[[164, 215], "R64"].sum(),
    "R32": coach_perf.loc[[164, 215], "R32"].sum(),
    "S16": coach_perf.loc[[164, 215], "S16"].sum(),
    "E8": coach_perf.loc[[164, 215], "E8"].sum(),
    "F4": coach_perf.loc[[164, 215], "F4"].sum(),
    "F2": coach_perf.loc[[164, 215], "F2"].sum(),
    "CHAMP": coach_perf.loc[[164, 215], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[164, 215], "TOP2"].sum(),
    "F4%": coach_perf.loc[[164, 215], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[164, 215], "CHAMP%"].mean(),
}

kermit_davis = {
    "COACH ID": min(coach_perf.iloc[54,0],coach_perf.iloc[193,0]),  # Keep the lower ID
    "COACH": "Kermit Davis",  # Corrected name
    "PAKE": coach_perf.loc[[54, 193], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[54, 193], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[54, 193], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[54, 193], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[54, 193], "GAMES"].sum(),
    "W": coach_perf.loc[[54, 193], "W"].sum(),
    "L": coach_perf.loc[[54, 193], "L"].sum(),
    "WIN%": coach_perf.loc[[54, 193], "WIN%"].mean(),
    "R64": coach_perf.loc[[54, 193], "R64"].sum(),
    "R32": coach_perf.loc[[54, 193], "R32"].sum(),
    "S16": coach_perf.loc[[54, 193], "S16"].sum(),
    "E8": coach_perf.loc[[54, 193], "E8"].sum(),
    "F4": coach_perf.loc[[54, 193], "F4"].sum(),
    "F2": coach_perf.loc[[54, 193], "F2"].sum(),
    "CHAMP": coach_perf.loc[[54, 193], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[54, 193], "TOP2"].sum(),
    "F4%": coach_perf.loc[[54, 193], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[54, 193], "CHAMP%"].mean(),
}

coach_perf = coach_perf.drop([54, 72, 164, 187, 193, 215, 291])

# Reset index
coach_perf = coach_perf.reset_index(drop=True)

# Add the new combined row
coach_perf = pd.concat([coach_perf, pd.DataFrame([steve_prohm,kermit_davis,bob_marlin])], ignore_index=True)

coach_perf

,COACH ID,COACH,PAKE,PAKE RANK,PASE,PASE RANK,GAMES,W,L,WIN%,R64,R32,S16,E8,F4,F2,CHAMP,TOP2,F4%,CHAMP%
0,1,Roy Williams,8.200000,1.000000,5.50,6.000000,44,34,10,0.7730,12,11,8,6,4,3,2,8,0.955000,0.5380
1,2,Tom Izzo,8.000000,2.000000,9.60,1.000000,47,32,15,0.6810,15,13,9,5,4,1,0,4,0.867000,0.3420
2,3,Brad Stevens,6.500000,3.000000,7.60,3.000000,17,12,5,0.7060,5,4,2,2,2,2,0,0,0.255000,0.0370
3,4,John Calipari,6.200000,4.000000,8.60,2.000000,52,39,13,0.7500,14,12,10,8,5,3,1,8,0.990000,0.7490
4,5,Dana Altman,6.100000,5.000000,6.70,4.000000,24,16,8,0.6670,8,8,5,2,1,0,0,1,0.431000,0.0820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,318,Rick Barnes,-6.400000,318.000000,-3.80,313.000000,28,15,13,0.5360,13,9,4,2,0,0,0,3,0.809000,0.2810
311,319,Tony Bennett,-6.500000,319.000000,-7.00,318.000000,24,15,9,0.6250,10,6,4,2,1,1,1,5,0.936000,0.5260
312,73,Steve Prohm,-0.466667,184.333333,-0.30,169.666667,8,4,4,0.5000,4,3,1,0,0,0,0,0,0.083333,0.0120
313,55,Kermit Davis,0.250000,124.500000,0.35,118.500000,5,2,3,0.4165,3,2,0,0,0,0,0,0,0.011000,0.0005


In [685]:
coach_perf['TEAMS'] = [['North Carolina'],['Michigan St.'],['Butler'],['Memphis','Kentucky','Arkansas'],['Creighton','Oregon'],['Syracuse'],['Michigan'],
                      ['Villanova'],['Nevada','Arkansas','USC'],['Connecticut'],['Loyola Chicago','Oklahoma'],['Kansas St.','South Carolina','Massachusetts'],
                      ['Wagner','Rhode Island','Connecticut'],['Kansas'],['George Mason','Miami FL'],['North Carolina'],['Gonzaga'],['Little Rock','Texas Tech','Texas','Missisippi'],
                      ['Florida'],['Connecticut',"Saint Joseph's"],["Saint Peter's",'Seton Hall'],['Florida Atlantic','Michigan'],['Louisville','Iona',"Saint John's"],['Duke'],
                      ['Xavier','Arizona','Xavier'],['Wisconsin'],['Ohio','Illinois','Akron'],['Dayton','Indiana','Rhode Island'],['San Diego St.'],
                      ['Oklahoma','Pittsburgh'],['Florida Gulf Coast','USC','SMU'],['Oral Robert','Wichita St.'],['Michigan'],['Stanford','UCF'],['Murray St.','Texas A&M'],['La Salle'],
                      ['Kansas St.'],['UNC Wilmington','North Carolina St.'],['Western Kentucky','South Carolina','Northern Kentucky'],['Princeton'],['Florida St.'],['Grand Canyon'],['Richmond'],
                      ['Montana','Oregon St.'],['Florida','Georgia'],['Buffalo','Alabama'],['Xavier','Louisville','College of Charleston'],['Alabama','North Carolina St.','Cal St. Northridge'],
                      ['Cornell','Boston College','Penn'],['Fairleigh Dickinson','Iona'],['Wright St.','Clemson'],['Northwestern'],['Norfolk St.'],['Mercer'],['UAB','Stanford'],
                      ['Yale'],['Marshall'],['UC Irvine'],['Morehead St.','Southern Miss','Tennessee'],['Western Kentucky'],['Abilene Christian','UTEP'],['Dayton','Georgia Tech','South Florida'],
                      ['San Diego'],['Furman'],['Duke'],['James Madison','Vanderbilt'],['Iowa St.','Creighton'],['Murray St.','LSU'],['Stony Brook','Ohio'],['Stanford','LSU','TCU'],
                      ['Northern Iowa'],['Lehigh'],['Alabama'],['Cleveland St.','Missouri'],['Washington'],['Arkansas'],
                      ['Hawaii'],['Cleveland St.'],['Georgia St.'],['Liberty','Liberty'],['Washington'],['Penn St.','Notre Dame'],['Oakland'],['Marquette','Virginia Tech','Texas A&M'],
                      ['Frenso St.','UTEP','Texas'],['Washington St.','Stanford'],['Saint Louis'],['North Dakota St.','Ohio'],['Georgia Tech','George Mason'],['Harvard'],['Baylor'],
                      ['Missouri','Arkansas',"Saint John's"],['Stony Brook','Rutgers'],['Montana St.','Utah St.','Washington'],['South Florida','Eastern Michigan'],['Utah'],
                      ['Chattanooga','Central Arkansas'],['UTSA'],['UT Arlington','Troy'],['Hampton'],['Mississippi St.','Western Kentucky'],['Arkansas Pine Bluff','Mississippi Valley St.'],
                      ['Lafayette'],['Southern Miss','Colorado St.'],['Mississippi Valley St.'],['Northwestern','Holy Cross'],['North Carolina A&T'],['Robert Morris'],['Radford'],
                      ['Grambling St.'],['Wagner'],['Marshall','UCF','Stetson'],['Fairleigh Dickinson'],['Bradley','UC Davis'],['Gardner Webb'],['Hartford','Manhattan'],
                      ['Radford','UNC Greensboro'],['North Dakota St.'],["Mount St. Mary's",'Siena','George Washington'],['Wagner',"Saint Peter's"],['UNC Asheville'],['Bradley'],
                      ['Norfolk St.'],['Georgia St.','SMU','Rice'],['Howard'],['Texas Southern'],['Army','Drexel'],['Stetson','Lipscomb','Belmont'],['Milwaukee','Butler'],
                      ['UMBC','Utah St.','VCU'],['North Dakota'],['Texas A&M','Maryland'],['Southern'],['Troy'],['Cal Poly'],['UNC Asheville','Middle Tennessee'],['Detroit'],
                      ['Cal St. Northridge'],['Binghamton','Morgan St.'],['James Madison'],['North Dakota St.'],['UMBC'],['Penn St.','Florida Gulf Coast'],['Northwestern St.'],
                      ['Boise St.'],['Austin Peay'],['Hampton'],['Loyola MD'],['Coastal Carolina'],["Mount St. Mary's",'Holy Cross'],['Northern Colorado'],['Milwaukee'],['Longwood'],
                      ['Delaware'],['Northern Kentucky','Cincinnati'],['Cal St. Fullerton'],['Stephen F. Austin'],['Notre Dame'],['Akron','Duquesne'],['Texas A&M Corpus Chris','Western Kentucky','Oklahoma St.'],
                      ['East Tennessee St.','Wake Forest'],['Maryland'],['Eastern Kentucky'],['East Tennesse St.'],['Cal St. Fullerton'],['Pacific'],['Princeton','Fairfield'],['Manhattan'],
                      ['Winthrop'],['Florida Southern','Green Bay'],['Eastern Washington'],['Georgia St.','Cal St. Bakersfield'],['Chattanooga','Massachusetts'],['UNC Asheville'],['Western Michigan'],
                      ['Indiana St.'],['Delaware','North Carolina A&T'],['Wyoming'],['UC Irvine'],['Robert Morris','Rutgers'],['Stephen F. Austin'],['Morgan St.'],["Saint Peter's"],['American'],
                      ['New Mexico St.','New Mexico'],['Charleston','Boston College'],['Kennesaw St.','South Florida'],['Oklahoma St.'],['Montana'],['Morehead St.'],['Tennessee','Auburn'],['Northeastern'],
                      ['Georgia','Cleveland St.'],['North Carolina Central'],['Army','Saint Louis'],['Houston'],['Tulsa','Wake Forest'],['Kentucky','Texas Tech','Ranger College','Tarleton St.'],['Portland St.','Washington St.'],
                      ['Weber St.'],['UC Santa Barbara'],['California'],['Long Island','Dusquesne','UMBC'],['Samford'],['Western Kentucky','Jacksonville St.'],['Eastern Washington','Portland'],['Bucknell','New Hampshire'],
                      ['Florida Gulf Coast','East Carolina'],['Midland College','Midwestern St.','Arkansas St.','North Texas','Texas Tech'],['UNC Greensboro','Cincinnati'],['American','Old Dominion'],
                      ['Boston College','Kennesaw St.'],['Georgetown'],['Oral Roberts'],['South Alabama'],['Massachusetts','LIU Brooklyn'],['Albany'],['UNLV','Oklahoma'],['Old Dominion'],['South Dakota St.'],['New Mexico','UCLA','Nevada'],
                      ['Mississippi','UAB'],['Indiana'],['Texas Tech'],['UC Santa Barbara'],['Kent St.'],['Kent St.','TCU','Ohio','Boston College'],['UTEP','Auburn','Central Michigan'],['Oregon','Washington St.'],
                      ['Wake Forest'],['Wisconsin'],['Minnesota','Long Beach St.','Eastern Washington'],['Loyola Chicago'],['Vermont','George Washington'],['USC'],['Penn St.','Navy'],['Marquette','Indiana','Georgia'],
                      ['Iona'],['South Dakota St.','UNLV','Iowa St.'],['North Texas','LSU','Texas Southern'],['Wichita St.'],['Bowling Green','New Mexico St.','Mississippi St.'],['Indiana'],['San Diego St.'],['UCLA','Mississippi St.'],
                      ['Furman','Drake','Colorado St.'],['Davidson'],['USC','UTEP'],["Saint Joseph's"],['Bucknell','George Mason','Holy Cross'],['Nevada','Georgia','California'],['Utah'],['Memphis'],['Rice','VCU','Penn St.'],
                      ['Colgate'],['SMU'],['Colorado St.','Nebraska','San Jose St.'],['New Mexico'],['Northen Colorado','Colorado'],['VCU','Alabama','Dayton'],['Chattanooga','South Carolina'],['South Dakota St.','Wright St.'],
                      ['Winthrop','Charleston','Louisville'],['FIU','Minnesota','New Mexico'],['Boise St.'],['Arizona St.','Santa Clara'],['West Virginia'],['Memphis','Georgia Tech'],['Fairfield','Providence','Georgetown'],['Valparaiso','Vanderbilt','Grand Canyon'],
                      ['South Dakota','Utah St.','Utah'],['Indiana','Houston'],["Saint John's",'San Diego'],['Drake','Providence','Central Michigan'],['Butler','Ohio St.','Depaul'],['Chattanooga','VCU','LSU','McNeese St.'],
                      ['Cincinnati','UCLA'],['Drake','West Virginia'],['Stephen F. Austin','Oklahoma St.','Illinois'],['New Mexico St.','UNLV'],['Marquette'],['Buffalo','Arizona St.'],['SMU'],['Iowa St.','Nebraska'],
                      ['St. Bonaventure'],['Vermont'],['Minnesota','Texas Tech','Memphis'],['UNLV'],['San Francisco','Florida'],['Tennessee','California','Missouri'],['Iona','Seton Hall','Maryland'],
                      ['Utah St.'],['Ohio St.','Butler'],['BYU','Kentucky'],['Illinois','Kansas St.'],['Belmont'],['Miami FL','Missouri','Tulsa'],['BYU'],['Purdue'],['Temple','La Salle'],
                      ['Siena','Iowa'],['Vanderbilt','Pittsburgh'],['Wofford','Virginia Tech'],['VCU','Texas','Marquette'],['Arizona'],['Oklahoma St.','Saint Louis'],['Clemson'],["Saint Mary's"],['Pittsburgh','TCU'],['Georgetown'],['Texas','Tennessee'],['Washington St.','Virginia']
                      ,['Murrary St.','Iowaw St.','Murrary St.'],['Middle Tennessee','Mississippi'],['Sam Houston St.','Louisiana Lafayette'],]
coach_perf['YRS'] = [['2003-2021'],['1995-2025'],['2007-2013'],['2000-2009','2009-2024','2024-2025'],['1994-2010','2010-2025'],['1976-2023'],['2007-2019'],
                     ['2001-2022'],['2015-2019','2019-2024','2024-2025'],['2012-2018'],['2011-2021','2021-2025'],['2007-2012','2012-2022','2022-2025'],
                     ['2010-2012','2012-2018','2018-2025'],['2003-2025'],['1997-2011','2011-2024'],['2021-2025'],['1999-2025'],['2015-2016','2016-2021','2021-2022','2023-2025'],
                     ['1996-2015'],['1986-2012','2018-2021'],['2018-2022','2022-2025'],['2018-2024','2024-2025'],['2001-2017','2020-2023','2024-2025'],['1980-2022'],
                     ['2004-2009','2009-2021','2022-2025'],['2001-2015'],['2008-2012','2012-2017','2017-2025'],['2011-2017','2017-2021','2022-2025'],['2017-2025'],
                     ['2006-2011','2018-2025'],['2011-2013','2013-2024','2024-2025'],['2017-2023','2023-2025'],['2019-2024'],['2008-2016','2016-2025'],['2006-2011','2011-2019'],['2004-2018'],
                     ['2022-2025'],['2014-2017','2017-2025'],['2003-2008','2008-2012','2019-2025'],['2011-2025'],['2002-2025'],['2009-2013'],['2005-2025'],
                     ['2006-2014','2014-2025'],['2015-2022','2022-2025'],['2015-2019','2019-2025'],['2009-2018','2018-2022','2024-2025'],['1998-2009','2011-2017','2018-2021'],
                     ['2000-2010','2010-2014','2015-2025'],['2022-2023','2023-2025'],['2006-2010','2010-2025'],['2013-2025'],['2007-2013'],['2008-2019'],['2012-2016','2016-2024'],
                     ['1999-2025'],['2014-2024'],['2010-2025'],['2006-2012','2012-2014','2014-2015'],['2008-2012'],['2011-2021','2021-2025'],['2003-2011','2011-2016','2017-2023'],
                     ['2007-2015'],['2017-2025'],['2022-2025'],['2020-2024','2024-2025'],['2006-2010','2010-2025'],['2015-2022','2022-2025'],['2016-2019','2019-2025'],['2004-2008','2008-2012','2012-2016'],
                     ['2006-2025'],['2007-2025'],['2015-2019'],['2019-2022','2022-2025'],['2017-2024'],['2007-2011'],
                     ['2015-2025'],['2006-2017'],['2011-2019'],['2007-2009','2015-2025'],['2002-2017'],['2021-2023','2023-2025'],['1984-2025'],['2008-2014','2014-2019','2019-2025'],
                     ['2011-2018','2018-2021','2022-2025'],['2019-2024','2024-2025'],['2007-2012'],['2007-2014','2014-2019'],['2000-2011','2011-2015'],['2007-2025'],['2003-2025'],
                     ['2006-2011','2011-2019','2019-2023'],['2005-2016','2016-2025'],['2019-2020','2023-2024','2024-2025'],['2007-2014','2021-2025'],['2011-2021'],
                     ['2004-2013','2024-2025'],['2006-2016'],['2006-2018','2019-2025'],['2009-2024'],['1998-2012','2016-2023'],['2008-2021','2022-2025'],
                     ['1995-2022'],['2004-2012','2012-2018'],['2005-2008'],['2000-2013','2015-2019'],['2012-2016'],['2010-2025'],['2007-2011'],
                     ['2017-2025'],['2022-2025'],['2007-2010','2010-2016','2019-2025'],['2013-2022'],['2002-2011','2011-2025'],['2013-2024'],['2010-2022','2023-2025'],
                     ['2011-2021','2021-2025'],['2014-2025'],['2012-2018','2018-2019','2019-2022'],['2012-2022','2022-2025'],['2018-2025'],['2015-2025'],
                     ['2014-2025'],['2019-2022','2022-2024','2024-2025'],['2019-2025'],['2012-2018'],['2009-2016','2016-2025'],['2011-2013','2013-2019','2019-2025'],['2016-2017','2017-2022'],
                     ['2016-2021','2021-2023','2023-2025'],['2006-2019'],['2007-2011','2011-2021'],['2011-2017'],['2013-2019'],['2009-2019'],['2013-2018','2018-2025'],['2008-2016'],
                     ['1996-2013'],['2007-2009','2019-2025'],['2008-2016'],['2014-2025'],['2004-2012'],['2011-2020','2022-2025'],['1999-2022'],
                     ['2002-2010'],['1990-2017'],['2009-2024'],['2004-2013'],['2007-2023'],['2003-2010','2010-2015'],['2010-2016'],['2005-2016'],['2018-2025'],
                     ['2016-2025'],['2015-2019','2019-2021'],['2013-2025'],['2016-2025'],['2000-2023'],['2004-2017','2017-2024'],['2021-2023','2023-2024','2024-2025'],
                     ['2015-2020','2020-2025'],['1989-2011'],['2005-2015'],['2003-2015'],['2003-2012'],['1988-2013'],['2007-2011','2011-2019'],['2011-2022'],
                     ['2007-2012'],['2006-2015','2015-2020'],['2011-2017'],['2007-2011','2011-2025'],['2015-2017','2017-2022'],['1996-2013'],['2003-2020'],
                     ['2010-2021'],['2006-2016','2023-2025'],['2011-2016'],['2010-2025'],['2007-2010','2010-2013'],['2000-2013'],['2006-2019'],['2006-2018'],['2013-2023'],
                     ['2016-2017','2017-2021'],['2014-2021','2021-2025'],['2019-2023','2023-2024'],['2017-2024'],['2014-2025'],['2016-2024'],['2005-2011','2014-2025'],['2006-2025'],
                     ['2003-2009','2017-2019'],['2009-2025'],['2002-2009','2012-2016'],['2004-2010'],['2012-2014','2014-2020'],['2007-2009','2011-2012','2015-2020','2020-2025'],['2005-2009','2009-2014'],
                     ['2006-2022'],['1998-2017'],['2008-2014'],['2002-2012','2012-2017','2021-2025'],['2020-2025'],['2012-2016','2016-2025'],['2017-2021','2021-2025'],['2015-2023','2023-2025'],
                     ['2013-2018','2018-2022'],['2004-2009','2016-2017','2017-2023','2023-2025'],['2011-2021','2021-2025'],['2000-2013','2013-2024'],
                     ['1997-2010','2015-2019'],['2017-2023'],['1999-2017'],['2007-2012'],['2008-2017','2017-2019'],['2002-2021'],['2004-2011','2011-2021'],['2001-2013'],['2019-2025'],['2007-2013','2013-2018','2019-2025'],
                     ['2006-2018','2020-2025'],['2021-2025'],['2021-2023'],['2017-2025'],['2011-2025'],['2002-2008','2008-2012','2012-2014','2014-2021'],['2006-2010','2010-2014','2021-2025'],['1997-2010','2014-2019'],
                     ['2007-2010'],['2015-2025'],['1999-2006','2007-2024','2024-2025'],['2021-2025'],['2005-2011','2011-2016'],['2009-2013'],['2003-2011','2011-2025'],['1999-2008','2008-2017','2018-2022'],
                     ['2010-2019'],['2016-2019','2019-2021','2021-2025'],['2001-2012','2012-2017','2018-2025'],['2007-2020'],['2014-2015','2017-2022','2022-2025'],['2007-2008'],['1999-2017'],['2003-2013','2015-2022'],
                     ['2013-2017','2017-2018','2018-2025'],['1989-2022'],['2005-2009','2010-2017'],['1995-2019'],['2008-2015','2015-2021','2023-2025'],['2004-2009','2009-2018','2019-2023'],['2007-2011'],['2018-2025'],['2014-2017','2017-2023','2023-2025'],
                     ['2011-2025'],['2012-2016'],['2007-2012','2012-2019','2021-2025'],['2013-2017'],['2006-2010','2010-2025'],['2006-2009','2009-2015','2017-2025'],['2017-2022','2022-2025'],['1995-2016','2016-2024'],
                     ['2012-2021','2021-2024','2024-2025'],['2012-2013','2013-2021','2021-2025'],['2010-2025'],['2006-2015','2016-2025'],['2007-2023'],['2009-2016','2016-2023'],['2006-2011','2011-2023','2023-2025'],['2011-2016','2016-2019','2020-2025'],
                     ['2014-2018','2018-2021','2021-2025'],['2006-2008','2014-2025'],['2010-2015','2022-2025'],['2007-2008','2008-2011','2012-2021'],['2014-2017','2017-2024','2024-2025'],['2013-2015','2015-2017','2017-2022','2023-2025'],
                     ['2006-2019','2019-2025'],['2018-2024','2024-2025'],['2013-2016','2016-2017','2017-2025'],['2007-2016','2016-2019'],['2014-2021'],['2013-2015','2015-2025'],['2016-2022'],['2010-2015','2019-2025'],
                     ['2007-2025'],['2011-2025'],['2007-2013','2013-2016','2016-2018'],['2011-2016'],['2019-2022','2022-2025'],['2011-2014','2014-2017','2017-2022'],['2007-2010','2010-2022','2022-2025'],
                     ['1998-2015'],['2004-2017','2022-2025'],['2019-2024','2024-2025'],['2003-2012','2012-2022'],['1986-2019'],['2004-2011','2011-2014','2014-2022'],['2005-2019'],['2005-2025'],['2006-2019','2022-2025'],
                     ['2005-2010','2010-2025'],['1999-2016','2016-2018'],['2002-2019','2019-2025'],['2009-2015','2015-2021','2021-2025'],['2021-2025'],['2008-2016','2016-2024'],['2003-2010'],['2001-2025'],['2003-2016','2016-2025'],['2004-2017'],['1998-2015','2015-2025'],['2006-2009','2009-2024']
                     ,['2011-2015','2015-2021','2022-2025'],['2002-2018','2018-2023'],['1998-2010','2010-2024']]


Created a column on who won each matchup for every game in the dataset. Will now add columns of interest to this dataset and will try to do feature engineering as to model who'd win each matchup.

In [730]:
wins = []
for i in range(0,len(team_matchups)-1,2):
    if team_matchups.iloc[i,8] > team_matchups.iloc[i+1,8]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)
    
comp_stats = team_matchups.iloc[:,[0,3,4,5,6,7,8]]
comp_stats['WIN'] = wins
        
comp_stats.head()

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/1728178133.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_stats['WIN'] = wins


,YEAR,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,WIN
0,2024,1067,Connecticut,1,1,64,91,1
1,2024,1026,Stetson,16,64,64,52,0
2,2024,1060,Florida Atlantic,8,64,64,65,0
3,2024,1036,Northwestern,9,32,64,77,1
4,2024,1029,San Diego St.,5,16,64,69,1


In [731]:
dfs = [comp_stats.drop(['SCORE'],axis=1),temp_away,temp_home,temp_neutral,temp_v_ranked,temp_mis,temp_his,team_stats]

comp_team_stats = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),dfs)

comp_team_stats = comp_team_stats.merge(temp_res,on='TEAM',how='left')

comp_team_stats.head()

KeyError: 'TEAM'

## Feature engineering

In [695]:
def win_matchup(df):
    matchup = []
    for i in range(0,len(df)-1,2):
        team_a = df.iloc[i,:]
        team_b = df.iloc[i+1,:]
    
        if team_a['WIN'] == 1:
            diff = team_a.iloc[7:]-team_b.iloc[7:]
            row = list(team_a.iloc[0:7]) + list(diff)
        else:
            diff = team_b.iloc[7:]-team_a.iloc[7:]
            row = list(team_b.iloc[0:7]) + list(diff)
        
        matchup.append(row)
        
    if len(df) % 2 != 0:
        final_a = df.iloc[len(df)-2, :]
        final_b = df.iloc[-1,:]
        
        if final_a['WIN'] == 1:
            diff = final_a.iloc[7:]-final_b.iloc[7:]
            row = list(final_a.iloc[0:7]) + list(diff)
        else:
            diff = final_b.iloc[7:]-final_a.iloc[7:]
            row = list(final_a.iloc[0:7]) + list(diff)
        matchup.append(row)

    columnss = list(df.columns)
    return pd.DataFrame(matchup,columns=columnss)

win_matchup(comp_team_stats)

,YEAR,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,WIN,BADJ O_A,BADJ D_A,WIN%_A,EFG%_A,EFG%D_A,PPPO_A,PPPD_A,BADJ O_H,BADJ D_H,WIN%_H,EFG%_H,EFG%D_H,PPPO_H,PPPD_H,BADJ O_N,BADJ D_N,WIN%_N,EFG%_N,EFG%D_N,PPPO_N,PPPD_N,TR RATING,V 1-25 WINS,V 1-25 LOSS,V 26-50 WINS,V 26-50 LOSS,LUCK RATING,CONSISTENCY TR RATING,AvgHeight,EffectiveHeight,Experience,Bench,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),NET RPI,RESUME,WAB RANK,ELO,Q1 W,Q2 W,Q3 Q4 L,PLUS 500,R SCORE,BADJ EM,FTR,FTRD,BADJ T,TOV%,TOV%D,OREB%,OP OREB%,RAW T,WAB,2PT%,2PT%D,3PT%,3PT%D,BLK%,BLKED%,AST%,OP AST%,2PTR,3PTR,2PTRD,3PTRD,ELITE SOS,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
0,2024,1067,Connecticut,1,1,64,1,23.5,-13.7,37.500000,6.7,-4.3,0.159,-0.043,13.7,-24.0,15.384615,0.9,-8.8,0.090,-0.216,5.8,-20.5,0.000000,-4.5,1.7,-0.058,-0.074,24.3,10,1,4,-1,-0.3,-1.9,0.74,0.19,0.64505,6.11,17.7,-23.2,0.2,0.0,-207,-210,-136,-156,12,8,-8,20,99.98,33.733333,4.433333,10.000000,-0.166667,-1.566667,2.966667,5.233333,-6.533333,0.766667,6.046667,3.133333,-6.033333,-1.533333,-0.233333,5.700000,-0.566667,12.900000,-1.233333,1.333333,-1.333333,7.433333,-7.433333,28.026667,10.8,13.3,9,6,5,5,5,4,4,2
1,2024,1036,Northwestern,9,32,64,1,-1.7,-10.3,-13.636364,-2.2,-0.5,-0.117,-0.023,3.4,2.1,-4.621849,0.1,5.1,-0.001,0.063,-13.3,-10.3,-27.777778,-8.3,-2.6,-0.148,-0.093,-1.0,2,4,2,3,1.6,0.7,1.04,0.75,-0.06373,-7.45,-1.9,-2.6,2.4,-0.2,14,-20,-7,17,2,-3,-2,-7,14.27,2.300000,-7.766667,11.466667,-4.633333,-1.566667,2.133333,-7.233333,-2.266667,-4.700000,0.103333,-7.800000,-0.300000,2.466667,1.100000,2.900000,3.100000,6.233333,18.133333,0.833333,-0.833333,-1.600000,1.600000,10.163000,-1.9,-1.9,1,2,-1,-1,-1,0,0,0
2,2024,1029,San Diego St.,5,16,64,1,-3.8,-13.0,-15.476190,-4.7,-2.0,-0.074,-0.077,1.5,-16.6,23.557692,4.7,-5.6,0.004,-0.172,0.1,-3.4,0.000000,-3.5,-3.3,0.015,-0.001,8.6,2,1,1,3,-3.0,-0.6,0.47,-0.03,0.88448,2.78,-0.3,-13.2,0.3,0.9,-85,-69,-61,-33,4,-1,-4,4,97.97,10.266667,0.966667,1.133333,-2.100000,-1.266667,1.200000,-2.166667,-1.733333,-1.766667,1.766667,0.200000,-1.900000,-2.133333,-4.366667,-0.833333,-2.133333,1.866667,-2.000000,-5.900000,5.900000,-2.566667,2.566667,11.940333,0.5,1.3,8,5,4,1,1,1,0,1
3,2024,1012,Yale,13,32,64,1,-6.5,4.7,10.000000,-1.7,4.6,-0.030,0.022,-9.4,13.1,-15.972222,2.8,8.6,-0.027,0.094,-22.9,0.3,-7.500000,-8.2,1.4,-0.211,-0.020,-14.3,-1,-2,-4,-2,2.3,-3.6,0.52,-0.36,-0.50175,-20.20,-9.5,9.4,2.7,-0.3,78,78,73,48,-3,-7,4,-9,-99.37,-18.966667,-10.733333,-13.600000,-5.233333,-0.866667,-1.933333,-5.100000,-7.033333,-5.300000,-2.673333,-2.633333,4.266667,-1.466667,3.933333,-5.200000,-0.900000,-6.966667,14.900000,1.866667,-1.866667,-8.166667,8.166667,-16.964333,2.3,1.1,-1,-2,-1,-1,-1,0,0,-1
4,2024,1062,Duquesne,11,32,64,1,-6.4,-1.7,15.454545,0.7,-0.5,-0.022,-0.055,-20.8,0.1,-13.888889,-8.5,0.0,-0.193,0.014,-8.4,-2.8,5.714286,-4.3,-0.1,-0.069,0.018,-10.7,-5,-3,0,-2,4.0,-1.7,-2.00,-1.09,0.47535,0.60,-12.6,-1.9,1.3,0.3,69,45,55,3,-4,1,3,-2,-98.00,-10.400000,7.400000,3.833333,-2.766667,1.900000,3.666667,-2.000000,4.966667,-3.233333,-1.533333,-6.966667,0.833333,-0.533333,-1.500000,5.300000,5.266667,-7.700000,5.300000,10.233333,-10.233333,-3.666667,3.666667,-11.203333,5.2,4.3,-7,-1,-1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,2008,13,UCLA,1,4,8,1,-1.1,-10.9,26.363640,1.7,-1.5,0.005,-0.114,1.4,-2.6,-4.117647,4.7,5.5,0.025,-0.022,-26.3,-14.5,20.000000,-20.8,-3.2,-0.260,-0.183,3.6,2,-3,-1,0,-0.6,-1.8,2.06,0.30,0.32000,5.44,-2.6,-7.4,NaN,NaN,-2,-2,-5,-6,0,3,0,6,0.10,0.666667,9.166667,-2.000000,0.833333,5.300000,5.000000,7.066667,-6.266667,-0.400000,0.800000,-0.100000,4.633333,-9.233333,-4.500000,-4.633333

## Model Testing